In [21]:
s={'a','b','c','f','d'}
print(sorted(s))
    

['a', 'b', 'c', 'd', 'f']


In [25]:
#!/home/tumbling20s/_Python/hospital/venv/bin/python3.7
from datetime import datetime,timedelta
from openpyxl import Workbook
from random import sample,randint

'''
Contiene la definicion de la clase Turnos 
'''

FESTIVOS = ('01-01','06-01','01-05','02-05','15-05','15-08','12-10','01-11','09-11','06-12','08-12','25-12')
DIAS_DE_VACACIONES_POR_PERS=5
PRINCIPIO='01-12-2019'
FIN='31-12-2019'


class turnos:
    def __init__(self, _vacaciones_x_pers, _desde, _hasta,_version=''):        
        self.festivos = {datetime.strptime(f+'-2019','%d-%m-%Y') for f in FESTIVOS}#cuentan como guardia doble
        #input
        self.medicos =  set(_vacaciones_x_pers.keys())
        self.vacaciones_x_pers = _vacaciones_x_pers
        #para saber cuanto lleva cada doctor si hacer una guardia
        self.todos_han_hecho_guardia = {"doble":False, #fines de semana y festivos
                                    "simple":False}#entre_semana
        self.ultima_guardia_x_pers={"doble":{}, #fines de semana y festivos
                                    "simple":{}}#entre_semana  
        #para programar_calendario
        self.calendario={}
        self.inicio = _desde
        self.final = _hasta
        self.dia = _desde
        self._tipo_de_guardia = self._tipo_de_dia()
        #para guardar el archivo
        self.version = _version
        
    def _tipo_de_dia(self):
        #identificar el tipo de guardia
        return 'doble' if self.dia.weekday() in (5,6) or self.dia in self.festivos else 'simple'    
    
    def programar_calendario(self):
        #Para cada día desde el principio hasta el final encontrar 4 medicos        
        while self.dia <= self.final:
            self.registrar_dia()
            self.dia += timedelta(days=1)  
            self._tipo_de_guardia = self._tipo_de_dia()
            
    def registrar_dia(self):
        los_4_medicos=set()                                    
        medicos_de_la_cola=set()
        # Buscar quien está librando ese día
        self.calendario[self.dia]={}
        self.calendario[self.dia]['Librando'] = self._de_vacaciones_o_salientes() # {'Vacaciones':vacaciones, 'Salientes':salientes}
        librando= self.calendario[self.dia]['Librando']['Vacaciones'].union(self.calendario[self.dia]['Librando']['Salientes'])
        #escoger primero los que todavía no hayan hecho una guardia de este tipo
        if self.todos_han_hecho_guardia[self._tipo_de_guardia]==False:
            medicos_de_la_cola= self._escoger_los_que_todavia_no_han_pasado(librando)
        if len(medicos_de_la_cola)<4:
            los_4_medicos= medicos_de_la_cola.union(self._escoger_medicos_descansados(4 - len(medicos_de_la_cola),
                                                                               librando))
        elif len(medicos_de_la_cola)==4:
            los_4_medicos=medicos_de_la_cola    
        # apunta las guardias
        for med in los_4_medicos:
            self.ultima_guardia_x_pers[self._tipo_de_guardia][med] = self.dia  
       
        self.calendario[self.dia]['De guardia']=los_4_medicos
        
                                           
    def _de_vacaciones_o_salientes(self):
        #Devolver la lista de medicos que están de vacaciones o salientes        
        vacaciones = {med for med in self.medicos if self.dia.strftime('%d-%m-%Y') in self.vacaciones_x_pers[med]}
        salientes  = {med for med in self.medicos 
                      if self.dia - timedelta(days=1) in (self.ultima_guardia_x_pers['doble'].get(med),self.ultima_guardia_x_pers['simple'].get(med))}
        return {'Vacaciones':vacaciones, 'Salientes':salientes}
    
    def _escoger_los_que_todavia_no_han_pasado(self,_librando):                                 
        #medicos disponibles con prioridad: todavía no han hecho este tipo de guardia y que no están de vacaciones
        cola_inicial = self.medicos.difference(set(self.ultima_guardia_x_pers[self._tipo_de_guardia].keys()))
        # Si la cola está vacía es que todos los medicos han hecho una guardia de este tipo ya
        if len(cola_inicial)==0:
            self.todos_han_hecho_guardia[self._tipo_de_guardia]=True
            print('Médicos en cola para guardia '+self._tipo_de_guardia+' ya han pasado todos!!')                                        
            return set() #{}                              
        disponibles = cola_inicial.difference(_librando)
        if len(disponibles)==0:
            print('Médicos '+ ', '.join(cola_inicial) +' estaban en cola para guardia '+self._tipo_de_guardia+' pero están de vacaciones')
        # mientras que no tengas 4 medicos y sobren disponibles añade uno hasta que no queden o que tengas 4
        los_medicos_que_no_han_pasado=set()
        while len(los_medicos_que_no_han_pasado)<4 and len(disponibles)>0:
            medico= sample(disponibles,1)[0]
            los_medicos_que_no_han_pasado.add(medico)
            disponibles.discard(medico) 
        print('Seleccionados '+ ', '.join(los_medicos_que_no_han_pasado)+' de la cola de médicos para guardia '+self._tipo_de_guardia)
        return los_medicos_que_no_han_pasado
                                        
    def _escoger_medicos_descansados(self, numero, librando):
        assert numero in (1,2,3,4)
        medicos_para_elegir= [(med,dia) for med,dia in self.ultima_guardia_x_pers[self._tipo_de_guardia].items()
                              if med not in librando]
        medicos_para_elegir.sort(key=lambda i:i[1])
        medicos_elegidos={medico[0] for medico in medicos_para_elegir[:numero]}
        return medicos_elegidos
    
    def imprimir_calendario(self):
        '''Crear un fichero excel con todos los datos input y output generados'''
        lista_de_medicos = list(self.medicos)
        wb=Workbook()
        ws=wb.active
        #crear la pestaña de vacaciones por persona con los encabezados de la tabla
        ws.title='Vacaciones Por Persona'
        ws.cell(row=1,column=1).value='Persona';ws.cell(row=1,column=2).value='Vacaciones escogidas'
        for num_med in range(1,len(lista_de_medicos)+1):            
            #por cada medico escribir su nombre y sus vacaciones
            ws.cell(row=num_med+1,column=1).value = 'Dra. '+str(num_med)
            lista_de_vacaciones=sorted(list(self.vacaciones_x_pers['Dra. '+str(num_med)]))
            for num_dia_libre in range(len(lista_de_vacaciones)):
                ws.cell(row=num_med+1,column= num_dia_libre+2).value= lista_de_vacaciones[num_dia_libre]
        #crear la pestaña de vacaciones por persona con los encabezados de la tabla
        ws=wb.create_sheet('Calendario')
        ws.cell(row=1,column=1).value='Fecha'
        ws.cell(row=1,column=2).value='De guardia'
        ws.cell(row=1,column=6).value='De vacaciones'
        lista_de_fechas=list(self.calendario.keys())
        lista_de_fechas.sort()
        for fecha in lista_de_fechas:
            #Por cada fecha escribir los 4 medicos de guardia y los que están de saliente o de vacaciones
            ws.cell(row=2+lista_de_fechas.index(fecha),column=1).value=fecha.strftime('%d-%m-%Y')
            medicos_de_guardia=tuple(sorted(self.calendario[fecha]['De guardia']))
            medicos_de_vacaciones=tuple(sorted(self.calendario[fecha]['Librando']['Vacaciones']))
#             medicos_de_saliente=tuple(self.calendario[fecha]['Librando']['Salientes'])
            for medico in medicos_de_guardia:                
                ws.cell(row=2+lista_de_fechas.index(fecha),column=2+medicos_de_guardia.index(medico)).value=medico
            for medico in medicos_de_vacaciones:
                ws.cell(row=2+lista_de_fechas.index(fecha),column=6+medicos_de_vacaciones.index(medico)).value=medico
#             for medico in medicos_de_saliente:
#                 ws.cell(row=2+lista_de_fechas.index(fecha),column=6).value='De saliente: '
#                 ws.cell(row=2+lista_de_fechas.index(fecha),column=15+medicos_de_saliente.index(medico)).value=medico
        filename='ejemplo'+str(self.version)+'.xlsx'
        wb.save(filename)        
            
        
 
def fecha_random(start, end):
    """Generate a random datetime between `start` and `end`"""
    return (start +timedelta(
        # Get a random amount of seconds between `start` and `end`
        seconds=randint(0, int((end - start).total_seconds()))
    )).strftime('%d-%m-%Y')


def _crear_vacaciones(_desde,_hasta):
    medicos={}
    vacaciones=set()
    for i in range(1,41):
        while len(vacaciones)<DIAS_DE_VACACIONES_POR_PERS:
            vacaciones.add(fecha_random(_desde,_hasta))
        medicos["Dra. "+str(i)]=vacaciones 
        vacaciones=set()
    return medicos

if __name__=='__main__':
    desde = input("¿cual es la fecha inicial?")  or PRINCIPIO
    hasta = input("¿cual es la fecha final?") or FIN
    desde,hasta =  datetime.strptime(desde,'%d-%m-%Y'), datetime.strptime(hasta,'%d-%m-%Y')
    for i in range(10):
        turnos_del_hospital=turnos(_crear_vacaciones(desde,hasta),desde,hasta,i) 
        turnos_del_hospital.programar_calendario()
        turnos_del_hospital.imprimir_calendario()      

¿cual es la fecha inicial?
¿cual es la fecha final?
Seleccionados Dra. 14, Dra. 22, Dra. 32, Dra. 20 de la cola de médicos para guardia doble
Seleccionados Dra. 39, Dra. 16, Dra. 4, Dra. 15 de la cola de médicos para guardia simple
Seleccionados Dra. 37, Dra. 28, Dra. 19, Dra. 34 de la cola de médicos para guardia simple
Seleccionados Dra. 11, Dra. 35, Dra. 31, Dra. 21 de la cola de médicos para guardia simple
Seleccionados Dra. 9, Dra. 40, Dra. 33, Dra. 18 de la cola de médicos para guardia simple
Seleccionados Dra. 17, Dra. 35, Dra. 19, Dra. 37 de la cola de médicos para guardia doble
Seleccionados Dra. 8, Dra. 13, Dra. 12, Dra. 2 de la cola de médicos para guardia doble
Seleccionados Dra. 29, Dra. 23, Dra. 10, Dra. 26 de la cola de médicos para guardia doble
Seleccionados Dra. 1, Dra. 8, Dra. 5, Dra. 24 de la cola de médicos para guardia simple
Seleccionados Dra. 29, Dra. 38, Dra. 32, Dra. 36 de la cola de médicos para guardia simple
Seleccionados Dra. 17, Dra. 3, Dra. 30, Dra. 12 d

Seleccionados Dra. 30, Dra. 12, Dra. 10, Dra. 33 de la cola de médicos para guardia doble
Seleccionados Dra. 9, Dra. 1, Dra. 16, Dra. 34 de la cola de médicos para guardia simple
Seleccionados Dra. 32, Dra. 13, Dra. 30, Dra. 35 de la cola de médicos para guardia simple
Seleccionados Dra. 6, Dra. 36, Dra. 10, Dra. 31 de la cola de médicos para guardia simple
Seleccionados Dra. 3, Dra. 40, Dra. 19, Dra. 18 de la cola de médicos para guardia simple
Seleccionados Dra. 6, Dra. 25, Dra. 31, Dra. 15 de la cola de médicos para guardia doble
Seleccionados Dra. 1, Dra. 22, Dra. 16, Dra. 4 de la cola de médicos para guardia doble
Seleccionados Dra. 9, Dra. 36, Dra. 19, Dra. 27 de la cola de médicos para guardia doble
Seleccionados Dra. 29, Dra. 24, Dra. 4, Dra. 12 de la cola de médicos para guardia simple
Seleccionados Dra. 8, Dra. 7, Dra. 28, Dra. 2 de la cola de médicos para guardia simple
Seleccionados Dra. 17, Dra. 14, Dra. 38, Dra. 25 de la cola de médicos para guardia simple
Seleccionados D

In [22]:
from pprint import pprint
d=turnos_del_hospital.ultima_guardia_x_pers[turnos_del_hospital._tipo_de_guardia]
l=turnos_del_hospital._las_guardias_mas_antiguas()
print(l)
print(turnos_del_hospital.dia)
print(turnos_del_hospital.calendario[turnos_del_hospital.dia]['Librando'])
for med in d.keys(): 
    if d[med] in l:
        pprint(med);pprint(d[med])

[datetime.datetime(2019, 11, 5, 0, 0), datetime.datetime(2019, 11, 6, 0, 0), datetime.datetime(2019, 11, 7, 0, 0), datetime.datetime(2019, 11, 8, 0, 0)]
2019-11-22 00:00:00
{'Vacaciones': {'Hippocrates 24', 'Hippocrates 15', 'Hippocrates 30', 'Hippocrates 37', 'Hippocrates 33', 'Hippocrates 21', 'Hippocrates 28', 'Hippocrates 2', 'Hippocrates 13'}, 'Salientes': {'Hippocrates 36', 'Hippocrates 24', 'Hippocrates 10', 'Hippocrates 16'}}
'Hippocrates 35'
datetime.datetime(2019, 11, 5, 0, 0)
'Hippocrates 4'
datetime.datetime(2019, 11, 6, 0, 0)
'Hippocrates 33'
datetime.datetime(2019, 11, 7, 0, 0)
'Hippocrates 15'
datetime.datetime(2019, 11, 8, 0, 0)


In [28]:
l=[o[1] for o in turnos_del_hospital.ultima_guardia_x_pers['doble'].items()]
l.sort()
print(l[:8])

[datetime.datetime(2019, 11, 1, 0, 0), datetime.datetime(2019, 11, 1, 0, 0), datetime.datetime(2019, 11, 1, 0, 0), datetime.datetime(2019, 11, 1, 0, 0), datetime.datetime(2019, 11, 2, 0, 0), datetime.datetime(2019, 11, 2, 0, 0), datetime.datetime(2019, 11, 2, 0, 0), datetime.datetime(2019, 11, 2, 0, 0)]


In [ ]:
from keyword import kwlist
print(kwlist)


In [78]:
 with open("input1.json",'r') as f:
        medicos_vacaciones = json.load(f) 
        print(medicos_vacaciones.keys())

dict_keys(['Dra.Eugelilla 1', 'Dra.Eugelilla 2', 'Dra.Eugelilla 3', 'Dra.Eugelilla 4', 'Dra.Eugelilla 5', 'Dra.Eugelilla 6', 'Dra.Eugelilla 7', 'Dra.Eugelilla 8', 'Dra.Eugelilla 9', 'Dra.Eugelilla 10', 'Dra.Eugelilla 11', 'Dra.Eugelilla 12', 'Dra.Eugelilla 13', 'Dra.Eugelilla 14', 'Dra.Eugelilla 15', 'Dra.Eugelilla 16', 'Dra.Eugelilla 17', 'Dra.Eugelilla 18', 'Dra.Eugelilla 19', 'Dra.Eugelilla 20', 'Dra.Eugelilla 21', 'Dra.Eugelilla 22', 'Dra.Eugelilla 23', 'Dra.Eugelilla 24', 'Dra.Eugelilla 25', 'Dra.Eugelilla 26', 'Dra.Eugelilla 27', 'Dra.Eugelilla 28', 'Dra.Eugelilla 29', 'Dra.Eugelilla 30', 'Dra.Eugelilla 31', 'Dra.Eugelilla 32', 'Dra.Eugelilla 33', 'Dra.Eugelilla 34', 'Dra.Eugelilla 35', 'Dra.Eugelilla 36', 'Dra.Eugelilla 37', 'Dra.Eugelilla 38', 'Dra.Eugelilla 39', 'Dra.Eugelilla 40'])


In [130]:
class person:
    def __init__(self, name, age):
        self.name=name
        self.age=age
        self.cryptokitties=[]
    def buy_a_cryptokitty(self,cryptokitty_id):
        self.last_cryptokitty_id=cryptokitty_id
        self.cryptokitties.append(last_cryptokitty_id)
    


In [162]:
datetime.datetime.strptime('01-05'+'-2019', '%d-%m-%Y')

datetime.datetime(2019, 5, 1, 0, 0)

In [4]:
from openpyxl import Workbook
wb=Workbook()
ws=wb.active
print(ws.title)
ws=wb.create_sheet('nueva')
print(ws.title)

Sheet
nueva


In [12]:
l=['a','b']
l.index('b')

1